In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import xarray as xr
import cfgrib
import pytz
import pygrib
import datetime
# Pandas Settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Estaciones

In [2]:
estaciones = gpd.read_file("..\Datos\Estaciones\Estaciones_AMB.shp",encoding='utf-8')
estaciones = estaciones[['CODI EOI','NOM ESTACI','LATITUD','LONGITUD']]

In [15]:
estaciones

,CODI EOI,NOM ESTACI,LATITUD,LONGITUD
0,8015021,Badalona,41.443985,2.237899
1,8019004,Barcelona (Poblenou),41.403880,2.204501
2,8019042,Barcelona (Sants),41.378780,2.133099
3,8019043,Barcelona (Eixample),41.385315,2.153800
4,8019044,Barcelona (Gràcia - Sant Gervasi),41.398724,2.153399
5,8019050,Barcelona (Ciutadella),41.386406,2.187398
6,8019054,Barcelona (Parc Vall Hebron),41.426110,2.148002
7,8019057,Barcelona (Palau Reial),41.387490,2.115200
8,8019058,Barcelona (Observatori Fabra),41.418430,2.123897
9,8089005,Gavà,41.303097,1.991498


# Procesamiento 2018-2019

In [11]:
gr1819 = xr.open_dataset('..\Datos\Meteorologia\Meteo_2018_2019_2.grib',engine='cfgrib')
#gr1819 = xr.open_dataset('..\Datos\Meteorologia\\Meteo_2018_2019.grib',engine='cfgrib')

In [12]:
meteoEstacion = pd.DataFrame([])
for i in range(estaciones.shape[0]):
    print(f'{i+1} - Procesando Estacion {estaciones.iloc[i,0]}')
    dsloc = gr1819.sel(latitude=estaciones.iloc[i,2],longitude=estaciones.iloc[i,3],method='nearest')
    df = dsloc.to_dataframe()
    # Insertar col de Cod EOI
    df.insert(loc=0,column="COD",value=estaciones.iloc[i,0])
    # Formatear df
    df.drop(columns=['number','surface'],inplace=True)
    df.sort_values(by=['valid_time'],inplace=True)
    df.reset_index(drop=True,inplace=True)
    # Almacenar en df meteoEstacion
    meteoEstacion = meteoEstacion.append(df)
meteoEstacion.reset_index(drop=True,inplace=True)

1 - Procesando Estacion 8015021
2 - Procesando Estacion 8019004
3 - Procesando Estacion 8019042
4 - Procesando Estacion 8019043
5 - Procesando Estacion 8019044
6 - Procesando Estacion 8019050
7 - Procesando Estacion 8019054
8 - Procesando Estacion 8019057
9 - Procesando Estacion 8019058
10 - Procesando Estacion 8089005
11 - Procesando Estacion 8101001
12 - Procesando Estacion 8125002
13 - Procesando Estacion 8125013
14 - Procesando Estacion 8157003
15 - Procesando Estacion 8169008
16 - Procesando Estacion 8169009
17 - Procesando Estacion 8194008
18 - Procesando Estacion 8196001
19 - Procesando Estacion 8205002
20 - Procesando Estacion 8211004
21 - Procesando Estacion 8245012
22 - Procesando Estacion 8252006
23 - Procesando Estacion 8263001
24 - Procesando Estacion 8263007
25 - Procesando Estacion 8301004


Borrado 31/12/2017 y 01/01/2020:

In [13]:
indNaN=meteoEstacion[meteoEstacion['valid_time'].dt.date.astype(str)=='2017-12-31'].index
indNaN2=meteoEstacion[meteoEstacion['valid_time'].dt.date.astype(str)=='2020-01-01'].index
meteoEstacion.drop(indNaN2,inplace=True)
meteoEstacion.drop(indNaN,inplace=True)
meteoEstacion.reset_index(drop=True,inplace=True)

Analisis de NA por estación:

In [89]:
meteoEstacion.groupby('COD').count()

,latitude,longitude,valid_time,u10,v10,t2m,ssrd,tp
COD,,,,,,,,
8015021,17520,17520,17520,0,0,0,0,0
8019004,17520,17520,17520,0,0,0,0,0
8019042,17520,17520,17520,17520,17520,17520,17520,17520
8019043,17520,17520,17520,0,0,0,0,0
8019044,17520,17520,17520,0,0,0,0,0
8019050,17520,17520,17520,0,0,0,0,0
8019054,17520,17520,17520,17520,17520,17520,17520,17520
8019057,17520,17520,17520,17520,17520,17520,17520,17520
8019058,17520,17520,17520,17520,17520,17520,17520,17520


Pasar grib a DF:

In [110]:
df1819 = gr1819.to_dataframe()
df1819.reset_index(drop=False,inplace=True)

In [109]:
latitud = df1819['latitude'].astype(str)
longitud = df1819['longitude']=df1819['longitude'].astype(str)

In [112]:
print(latitud.unique())
print(longitud.unique())

['42.0' '41.9' '41.8' '41.699999999999996' '41.599999999999994'
 '41.49999999999999' '41.39999999999999' '41.29999999999999'
 '41.19999999999999' '41.09999999999999' '40.999999999999986']
['1.5' '1.6' '1.7000000000000002' '1.8000000000000003'
 '1.9000000000000004' '2.0000000000000004' '2.1000000000000005'
 '2.2000000000000006' '2.3000000000000007' '2.400000000000001'
 '2.500000000000001' '2.600000000000001' '2.700000000000001'
 '2.800000000000001' '2.9000000000000012' '3.0000000000000013']


In [74]:
df414_22 = df1819[(df1819['latitude']=='41.39999999999999') & (df1819['longitude']=='2.2000000000000006')].head()
indNaN=df414_22[df414_22['valid_time'].dt.date.astype(str)=='2017-12-31'].index
indNaN2=df414_22[df414_22['valid_time'].dt.date.astype(str)=='2020-01-01'].index
df414_22.drop(indNaN2,inplace=True)
df414_22.drop(indNaN,inplace=True)
df414_22.reset_index(drop=True,inplace=True)

In [97]:
df1819[(df1819['latitude']=='41.39999999999999') & (df1819['longitude']=='2.2000000000000006')].count()

latitude      17544
longitude     17544
step          17544
time          17544
number        17544
surface       17544
valid_time    17544
u10               0
v10               0
t2m               0
ssrd              0
tp                0
dtype: int64

NO HAY DATOS EN LA CELDA de LAT = 41.4 y LONG = 2.2 o 2.3. Deja sin datos a las estaciones:
 
 - 8015021 - Badalona --> 41.45 - 2.2
 - 8019004 - Poblenou --> 41.40 - 2.2
 - 8019043 - Eixample --> 41.38 - 2.15
 - 8019044 - Gracia --> 41.39 - 2.15
 - 8019050 - Ciudadela --> 41.38 - 2.18
 - 8194008 - Sant Adriá del Besos --> 41.43 - 2.22
 - 8245012 - Santaco --> 41.45 - 2.21
 

Introducimos datos de diferentes barrio a los barrios que no tienen datos:

   - Sants : 8019042
   - Vall d'Hebron : 8019054
   - Observatorio Fabra : 8019058

Comprobamos coordenadas costeras para introducir en la cornisa mediterranea:

In [113]:
df43_21 = df1819[(df1819['latitude']==41.29999999999999) & (df1819['longitude']==2.1000000000000005)]
indNaN=df43_21[df43_21['valid_time'].dt.date.astype(str)=='2017-12-31'].index
indNaN2=df43_21[df43_21['valid_time'].dt.date.astype(str)=='2020-01-01'].index
df43_21.drop(indNaN2,inplace=True)
df43_21.drop(indNaN,inplace=True)
df43_21.reset_index(drop=True,inplace=True)

C:\Users\iarregui\anaconda3\envs\geo_env\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [120]:
df43_21.head(5)

,latitude,longitude,step,time,number,surface,valid_time,u10,v10,t2m,ssrd,tp
0,41.3,2.1,0 days 01:00:00,2018-01-01,0,0.0,2018-01-01 01:00:00,6.111679,-4.243378,285.035156,0.0,0.0
1,41.3,2.1,0 days 01:00:00,2018-01-02,0,0.0,2018-01-02 01:00:00,4.464066,-0.736460,285.090088,0.0,0.0
2,41.3,2.1,0 days 01:00:00,2018-01-03,0,0.0,2018-01-03 01:00:00,6.251740,-4.920866,286.806885,0.0,0.0
3,41.3,2.1,0 days 01:00:00,2018-01-04,0,0.0,2018-01-04 01:00:00,7.848984,-3.121474,288.958740,0.0,0.0
4,41.3,2.1,0 days 01:00:00,2018-01-05,0,0.0,2018-01-05 01:00:00,4.564117,-1.370241,286.538818,0.0,0.0


Se introducen los datos de Sants en Gracia y Eixample y df4321 en Badalona, Poblenou, Ciudadela, Sant Adria de Besos y Santaco

In [138]:
indBad = meteoEstacion[meteoEstacion['COD']==8015021].index
indSants = meteoEstacion[meteoEstacion['COD']==8019042].index
indEix = meteoEstacion[meteoEstacion['COD']==8019043].index
indPal = meteoEstacion[meteoEstacion['COD']==8019057].index
indGra = meteoEstacion[meteoEstacion['COD']==8019044].index
indPob = meteoEstacion[meteoEstacion['COD']==8019004].index
indCiud = meteoEstacion[meteoEstacion['COD']==8019050].index
indValHeb = meteoEstacion[meteoEstacion['COD']==8019054].index
indFabra = meteoEstacion[meteoEstacion['COD']==8019058].index
indSantAd = meteoEstacion[meteoEstacion['COD']==8194008].index
indSantaco = meteoEstacion[meteoEstacion['COD']==8245012].index

In [139]:
meteoEstacion.iloc[indEix,4:9] = meteoEstacion.iloc[indSants,4:9]
meteoEstacion.iloc[indGra,4:9] = meteoEstacion.iloc[indSants,4:9]
meteoEstacion.iloc[indBad,4:9] = df43_21.iloc[:,7:12]
meteoEstacion.iloc[indPob,4:9] = df43_21.iloc[:,7:12]
meteoEstacion.iloc[indCiud,4:9] = df43_21.iloc[:,7:12]
meteoEstacion.iloc[indSantAd,4:9] = df43_21.iloc[:,7:12]
meteoEstacion.iloc[indSantaco,4:9] = df43_21.iloc[:,7:12]

In [140]:
meteoEstacion.groupby('COD').count()

,latitude,longitude,valid_time,u10,v10,t2m,ssrd,tp
COD,,,,,,,,
8015021,17520,17520,17520,17520,17520,17520,17520,17520
8019004,17520,17520,17520,17520,17520,17520,17520,17520
8019042,17520,17520,17520,17520,17520,17520,17520,17520
8019043,17520,17520,17520,17520,17520,17520,17520,17520
8019044,17520,17520,17520,17520,17520,17520,17520,17520
8019050,17520,17520,17520,17520,17520,17520,17520,17520
8019054,17520,17520,17520,17520,17520,17520,17520,17520
8019057,17520,17520,17520,17520,17520,17520,17520,17520
8019058,17520,17520,17520,17520,17520,17520,17520,17520


In [141]:
meteoEstacion.to_csv('..\Datos\Datos_Procesados\MeteoEstaciones_1819.csv')

In [ ]:
'''ds = cfgrib.open_dataset('..\Datos\Meteorologia\Meteo_2018_2019.grib')
ds_df=ds.to_dataframe()
ds_df.reset_index(drop=False,inplace=True)
ds_df.to_csv("MeteoERA5_2018-2019.csv")'''

In [142]:
del meteoEstacion

# Procesamiento 2020-2021

Cargamos el grib:

In [143]:
gr2021 = xr.open_dataset('..\Datos\Meteorologia\\meteo_2020_2021.grib',engine='cfgrib')

Generación datos para cada estación:

In [144]:
meteoEstacion = pd.DataFrame([])
for i in range(estaciones.shape[0]):
    print(f'{i+1} - Procesando Estacion {estaciones.iloc[i,0]}')
    dsloc = gr2021.sel(latitude=estaciones.iloc[i,2],longitude=estaciones.iloc[i,3],method='nearest')
    df = dsloc.to_dataframe()
    # Insertar col de Cod EOI
    df.insert(loc=0,column="COD",value=estaciones.iloc[i,0])
    # Formatear df
    df.drop(columns=['number','surface'],inplace=True)
    df.sort_values(by=['valid_time'],inplace=True)
    df.reset_index(drop=True,inplace=True)
    # Almacenar en df meteoEstacion
    meteoEstacion = meteoEstacion.append(df)
meteoEstacion.reset_index(drop=True,inplace=True)

1 - Procesando Estacion 8015021
2 - Procesando Estacion 8019004
3 - Procesando Estacion 8019042
4 - Procesando Estacion 8019043
5 - Procesando Estacion 8019044
6 - Procesando Estacion 8019050
7 - Procesando Estacion 8019054
8 - Procesando Estacion 8019057
9 - Procesando Estacion 8019058
10 - Procesando Estacion 8089005
11 - Procesando Estacion 8101001
12 - Procesando Estacion 8125002
13 - Procesando Estacion 8125013
14 - Procesando Estacion 8157003
15 - Procesando Estacion 8169008
16 - Procesando Estacion 8169009
17 - Procesando Estacion 8194008
18 - Procesando Estacion 8196001
19 - Procesando Estacion 8205002
20 - Procesando Estacion 8211004
21 - Procesando Estacion 8245012
22 - Procesando Estacion 8252006
23 - Procesando Estacion 8263001
24 - Procesando Estacion 8263007
25 - Procesando Estacion 8301004


Eliminar 31/12/2019:

In [145]:
indNaN=meteoEstacion[meteoEstacion['valid_time'].dt.date.astype(str)=='2019-12-31'].index
meteoEstacion.drop(indNaN,inplace=True)
meteoEstacion.reset_index(drop=True,inplace=True)

Contar nulos por estacion:

In [146]:
meteoEstacion.groupby('COD').count()

,latitude,longitude,valid_time,u10,v10,t2m,ssrd,tp
COD,,,,,,,,
8015021,10201,10201,10201,0,0,0,0,0
8019004,10201,10201,10201,0,0,0,0,0
8019042,10201,10201,10201,10201,10201,10201,10201,10201
8019043,10201,10201,10201,0,0,0,0,0
8019044,10201,10201,10201,0,0,0,0,0
8019050,10201,10201,10201,0,0,0,0,0
8019054,10201,10201,10201,10201,10201,10201,10201,10201
8019057,10201,10201,10201,10201,10201,10201,10201,10201
8019058,10201,10201,10201,10201,10201,10201,10201,10201


Generamos df para lat=4.3 y long=2.1:

In [147]:
df2021 = gr2021.to_dataframe()
df2021.reset_index(drop=False,inplace=True)

In [150]:
df43_21 = df2021[(df2021['latitude']==41.29999999999999) & (df2021['longitude']==2.1000000000000005)]
indNaN=df43_21[df43_21['valid_time'].dt.date.astype(str)=='2019-12-31'].index
df43_21.drop(indNaN,inplace=True)
df43_21.reset_index(drop=True,inplace=True)

C:\Users\iarregui\anaconda3\envs\geo_env\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Generar indices de cada estacion necesaria:

In [152]:
indBad = meteoEstacion[meteoEstacion['COD']==8015021].index
indSants = meteoEstacion[meteoEstacion['COD']==8019042].index
indEix = meteoEstacion[meteoEstacion['COD']==8019043].index
indPal = meteoEstacion[meteoEstacion['COD']==8019057].index
indGra = meteoEstacion[meteoEstacion['COD']==8019044].index
indPob = meteoEstacion[meteoEstacion['COD']==8019004].index
indCiud = meteoEstacion[meteoEstacion['COD']==8019050].index
indValHeb = meteoEstacion[meteoEstacion['COD']==8019054].index
indFabra = meteoEstacion[meteoEstacion['COD']==8019058].index
indSantAd = meteoEstacion[meteoEstacion['COD']==8194008].index
indSantaco = meteoEstacion[meteoEstacion['COD']==8245012].index

Rellanar datos de estaciones con Nulos con los siguientes datos:
 - Sants -> Eixample, Gracia
 - df43:21 -> Badalona, Poblenou, Ciudadela, Sant Adria Besos, Santaco

In [153]:
meteoEstacion.iloc[indEix,4:9] = meteoEstacion.iloc[indSants,4:9]
meteoEstacion.iloc[indGra,4:9] = meteoEstacion.iloc[indSants,4:9]
meteoEstacion.iloc[indBad,4:9] = df43_21.iloc[:,7:12]
meteoEstacion.iloc[indPob,4:9] = df43_21.iloc[:,7:12]
meteoEstacion.iloc[indCiud,4:9] = df43_21.iloc[:,7:12]
meteoEstacion.iloc[indSantAd,4:9] = df43_21.iloc[:,7:12]
meteoEstacion.iloc[indSantaco,4:9] = df43_21.iloc[:,7:12]

Comprobar que no hay nulos:

In [154]:
meteoEstacion.groupby('COD').count()

,latitude,longitude,valid_time,u10,v10,t2m,ssrd,tp
COD,,,,,,,,
8015021,10201,10201,10201,10201,10201,10201,10201,10201
8019004,10201,10201,10201,10201,10201,10201,10201,10201
8019042,10201,10201,10201,10201,10201,10201,10201,10201
8019043,10201,10201,10201,10201,10201,10201,10201,10201
8019044,10201,10201,10201,10201,10201,10201,10201,10201
8019050,10201,10201,10201,10201,10201,10201,10201,10201
8019054,10201,10201,10201,10201,10201,10201,10201,10201
8019057,10201,10201,10201,10201,10201,10201,10201,10201
8019058,10201,10201,10201,10201,10201,10201,10201,10201


In [155]:
meteoEstacion.to_csv('..\Datos\Datos_Procesados\MeteoEstaciones_2021.csv')